In [ ]:
# import python

import csv
import matplotlib.pylab as plt
import numpy as np
import os
import pandas as pd
import scipy.interpolate as spi

## Emille's code

In [ ]:
# import R
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter
import rpy2.robjects as ro

In [ ]:
# import R packages

ro.r('library(spatstat)')
ro.r('library(sparr)')
ro.r('library(dplyr)')
ro.r('library(magrittr)')
ro.r('library(reshape2)')

In [ ]:
# read variables into R section
ro.r('h0 = 0.01')
ro.r('resolution = 256')

# hear I am reading 1/64 of the data I separated myself
ro.r('dataRAW <- read.csv("data/64split/rawdata_00_00_00.csv")')  

In [ ]:
# get single columns from data frame
ro.r('x <- dataRAW$RA')
ro.r('y <- dataRAW$DEC')

# put in the correct format
ro.r('myPointData <- ppp(x, y, xrange=range(x), yrange=range(y))')

# do KDE
ro.r('ddest <- bivariate.density(myPointData, h0=h0, adapt=TRUE, resolution=resolution)')

In [ ]:
# output from KDE
ro.r('myDensEst <- ddest$z')

# define boundaries
ro.r('mins <- c(min(x), min(y))')
ro.r('maxs <- c(max(x), max(y))')

In [ ]:
# number of points to sample
ro.r('nPoints = 520000')

# normalize
ro.r('densEst <- ddest$kde2d/max(ddest$kde2d, na.rm=TRUE)')
     
# get dimension
ro.r('xdim <- dim(densEst)[1]')
ro.r('ydim <- dim(densEst)[2]')

ro.r('myNewXX <- vector(mode="numeric", length=nPoints)')
ro.r('myNewYY <- vector(mode="numeric", length=nPoints)')
ro.r('selPoints <- 1')


In [ ]:
data_raw = pd.read_csv('data/64split/rawdata_00_00_00.csv')

# Sample from density
for i in range(520000):
    ro.r('xx <- (runif(1) * xdim)')
    ro.r('yy <- (runif(1) * ydim)')
    ro.r('zz <- runif(1)')
    ro.r('myNewXX[selPoints] <- xx')
    ro.r('myNewYY[selPoints] <- yy')
    ro.r('selPoints <- selPoints + 1')

In [ ]:
# get new points into dataframe
ro.r('dataOUT = data.frame(x=myNewXX, y=myNewYY)')

In [ ]:
# write output to file
ro.r('write.csv(dataOUT, "from_KDE_00_00_00.csv")')

## Alex's code

In [ ]:
# read output in python
particles = pd.read_csv('../Data/64split/rawdata_00_00_00.csv')

In [ ]:
len(particles)

In [ ]:
# read output in python
galaxies = pd.read_csv('../Data/from_KDE/dens_00_00_00.csv')

In [ ]:
ra_grid = galaxies['RA'].unique()
dec_grid = galaxies['DEC'].unique()
dens_grid = np.reshape(galaxies['Dens'].values, (len(ra_grid), len(dec_grid)))

In [ ]:
galaxy_dens = spi.RectBivariateSpline(ra_grid, dec_grid, dens_grid)

In [ ]:
plt.figure(figsize=(20,10))

plt.subplot(1,2,1)
plt.title('original')
plt.scatter(particles['RA'], particles['DEC'], s=.01)

plt.subplot(1,2,2)
plt.title('interpolated KDE')
plt.contourf(ra_grid, dec_grid, dens_grid.T)
plt.show()

In [ ]:
plt.hist(dens_grid.flatten())

In [ ]:
print(dens_grid.mean())
cut_area = (extrema['RA']['max'] - extrema['RA']['min']) * (extrema['DEC']['max'] - extrema['DEC']['min'])
assert(np.isclose(1., np.sum(dens_grid) * cut_area / 256**2))

placeholder for global mean necessary for bias, actually depends on total angular area at that redshift

In [ ]:
n_SLICS = 3.6e9 / (64 * 64 * cut_area * 256**2)
mean_dens_SLICS = n_SLICS / (cut_area)
print(n_SLICS)

In [ ]:
bias_placeholder = 0.1 * n_SLICS

In [ ]:
# still working on this
def samp_gal_pos(cells, threshold):
    np.random.randint(0, len(cells))
def rej_samp(dens, threshold):
    if dens > threshold:
        

In [ ]:
extrema = particles.agg([min, max])
mock_samps = pd.DataFrame.from_dict({'RA': [], 'DEC': []})
mock_samps['RA'] = np.random.uniform(extrema['RA']['min'], extrema['RA']['max'], 1000)
mock_samps['DEC'] = np.random.uniform(extrema['DEC']['min'], extrema['DEC']['max'], 1000)